In [23]:

# use documents to train topic model
import requests
import json
api_key = '0jXJPFqzCeDSA1WNFyAjbG0Q7hJMyZsK' # Replace with your own API key
url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=news&api-key={api_key}&sort=newest'

response = requests.get(url)
data = response.json()
# extract article text and add to list of documents
documents = []
if response.status_code == 200:
    articles = response.json()["response"]["docs"]
    for article in articles:
        text = article["abstract"] + " " + article["lead_paragraph"] + " " + article["snippet"] + " " + article["headline"]["main"]
        documents.append(text)
else:
    print("Error getting articles")

print (json.dumps(documents, indent=4))




[
    "Returning to activities you\u2019ve given up can be a surprising source of confidence and wonder. There are many reasons not to ski, and I\u2019ve availed myself of most of them over the past two decades when asked why I gave up an activity I\u2019d once loved. Cost was high on the list, followed by inconvenience, cold and risk to life and limb. Returning to activities you\u2019ve given up can be a surprising source of confidence and wonder. Getting Back on Skis",
    "A winter of gyrating temperatures and rainfalls caused officials to give up on the \u201cworld\u2019s largest skating rink.\u201d While people in the Ottawa area were reluctant to admit it, the announcement on Friday was widely expected: For the first time since its creation 52 years ago, the Rideau Canal Skateway, which bills itself as the \u201cworld\u2019s largest skating rink,\u201d will not open this season. A winter of gyrating temperatures and rainfalls caused officials to give up on the \u201cworld\u2019s 

In [21]:
import json
import requests

# API endpoint for retrieving New York Times articles
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"

# Parameters for retrieving articles
query = "politics"  # Example query
num_articles = 100  # Number of articles to retrieve

# Retrieve articles from API
params = {"q": query, "api-key": api_key, "page": 0}
articles = []
while len(articles) < num_articles:
    response = requests.get(url, params=params)
    data = json.loads(response.text)
    articles.extend(data["response"]["docs"])
    params["page"] += 1

# Extract article text and label
train_docs = []
train_labels = []
for article in articles:
    text = article["abstract"]  # Use abstract as an example
    label = query  # Use query as label
    train_docs.append(text)
    train_labels.append(label)

# Print the first 5 train_docs and train_labels
print("Train Docs:\n", train_docs[:5])
print("Train Labels:\n", train_labels[:5])


Train Docs:
 ['The Congressional Dads Caucus makes savvy use of its own faint goofiness.', 'The backtracking by the Florida Republican came after bipartisan criticism and signaled how completely the G.O.P. has pulled away from calls to overhaul the nation’s entitlement programs.', 'The experience of unfreedom matters when we consider American democracy.', 'Her plans for a “national divorce” are very revealing.', 'J.D. Vance, the freshman senator from Ohio, is in the spotlight for the first time in his tenure as he responds to the train derailment and its aftermath.']
Train Labels:
 ['politics', 'politics', 'politics', 'politics', 'politics']


In [22]:
import re
import numpy as np

# Define the training data
# train_docs = [
#     'car vehicle drive road',
#     'music song band listen',
#     'food eat restaurant dish',
#     'movie watch film actor'
# ]

# train_labels = [
#     'cars',
#     'music',
#     'food',
#     'movies'
# ]

# Define a function to preprocess text
def preprocess(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove non-alphabetic characters
    text = re.sub('[^A-Za-z]+', ' ', text)
    
    # Split into words
    words = text.split()
    
    return words

# Define a function to generate a bag of words from a document
def bag_of_words(text, vocab):
    # Preprocess the text
    words = preprocess(text)
    
    # Initialize the bag with zeros
    bag = np.zeros(len(vocab))
    
    # Count the frequency of each word in the text
    for word in words:
        if word in vocab:
            bag[vocab.index(word)] += 1
    
    return bag

# Define a function to train a topic classification model
def train_model(docs, labels):
    # Create a vocabulary of unique words
    vocab = []
    for doc in docs:
        words = preprocess(doc)
        for word in words:
            if word not in vocab:
                vocab.append(word)
    
    # Generate bags of words for each document
    bags = []
    for doc in docs:
        bag = bag_of_words(doc, vocab)
        bags.append(bag)
    
    # Convert the bags to a numpy array
    bags = np.array(bags)
    
    # Create a one-hot encoding of the labels
    num_labels = len(labels)
    label_encoding = np.zeros((num_labels, num_labels))
    np.fill_diagonal(label_encoding, 1)
    
    # Train a linear regression model
    model = np.linalg.lstsq(bags, label_encoding, rcond=None)[0]
    
    return model, vocab

# Train the model
model, vocab = train_model(train_docs, train_labels)

# Define a function to predict the topic of a new document
def predict_topic(text, model, vocab):
    # Generate a bag of words for the new document
    bag = bag_of_words(text, vocab)
    
    # Predict the label using the model
    label_encoding = np.dot(bag, model)
    label_index = np.argmax(label_encoding)
    label = train_labels[label_index]
    
    return label

# Test the model
test_docs = [
    'food and restaurant reviews',
    'new car models for 2022',
    'the latest music album releases',
    'hollywood movie news'
]

for doc in test_docs:
    topic = predict_topic(doc, model, vocab)
    print(doc, '->', topic)


food and restaurant reviews -> politics
new car models for 2022 -> politics
the latest music album releases -> politics
hollywood movie news -> politics
